## Summary

- [groupby](#groupby)
- [group by count distinct](#group-by-count-distinct)
- [transform 操作](#transform-操作)
- [apply](#apply)
- [获取分组里最大值所在的行方法](#获取分组里最大值所在的行方法)
- [组内排序](#组内排序)

### group by count distinct
SQL:

```SQL
SELECT count(distinct M) FROM table GROUP BY K;
```

在pandas:

```python
df.groupby('K').M.nunique()
```

In [1]:
import pandas as pd
import numpy as np

In [63]:
df = pd.DataFrame({
    'id': [1, 1, 2, 3, 3, 3, 4],
    'name': ['a', 'a', 'b', np.nan, 'a', 'a', np.nan]
})
df

,id,name
0,1,a
1,1,a
2,2,b
3,3,NaN
4,3,a
5,3,a
6,4,NaN


In [64]:
df.groupby('name').id.nunique()

name
a    2
b    1
Name: id, dtype: int64

## groupby

1. 首先通过groupby得到DataFrameGroupBy对象, 比如data.groupby('race')
2. 然后选择需要研究的列, 比如`['age']`, 这样我们就得到了一个SeriesGroupby, 它代表每一个组都有一个Series
3. 对SeriesGroupby进行操作, 比如.mean(), 相当于对每个组的Series求均值

**总结**：
Groupby 可以简单总结为 split, apply, combine, 也就是说:

- split : 先将数据按一个属性分组 (得到 DataFrameGroupby / SeriesGroupby )
- apply : 对每一组数据进行操作 (取平均 取中值 取方差 或 自定义函数)
- combine: 将操作后的结果结合起来 (得到一个DataFrame 或 Series 或可视化图像)

docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [62]:
# 读入一个数据集, 我使用了美国警方击毙数据集.
path = 'https://raw.githubusercontent.com/HoijanLai/dataset/master/PoliceKillingsUS.csv'
data = pd.read_csv(path, encoding ='latin1')
data.sample(3)

,name,date,race,age,signs_of_mental_illness,flee
2363,Francisco Suarez-Madonado,01/06/17,H,27.0,True,Not fleeing
406,Edelmiro Hernandez,03/06/15,H,33.0,False,Car
2305,Jayson Negron,09/05/17,H,16.0,False,Car


In [65]:
data.groupby('race')['age'].mean() # 不同种族年龄均值

race
A    36.605263
B    31.635468
H    32.995157
N    30.451613
O    33.071429
W    40.046980
Name: age, dtype: float64

In [66]:
# .unstack() 展开序列
# 有心理疾病的种族分组
data.groupby('race')['signs_of_mental_illness'].value_counts().unstack()

signs_of_mental_illness,False,True
race,,
A,29,10
B,523,95
H,338,85
N,23,8
O,21,7
W,819,382


In [67]:
agg_df = data.groupby('race').agg({'age': [np.median, np.mean], 'signs_of_mental_illness': np.mean})
agg_df

age            signs_of_mental_illness
     median       mean                    mean
race                                          
A      35.0  36.605263                0.256410
B      30.0  31.635468                0.153722
H      31.0  32.995157                0.200946
N      29.0  30.451613                0.258065
O      29.5  33.071429                0.250000
W      38.0  40.046980                0.318068

In [68]:
# 👍👍👍👍 拍平
agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
agg_df

,age_median,age_mean,signs_of_mental_illness_mean
race,,,
A,35.0,36.605263,0.256410
B,30.0,31.635468,0.153722
H,31.0,32.995157,0.200946
N,29.0,30.451613,0.258065
O,29.5,33.071429,0.250000
W,38.0,40.046980,0.318068


In [29]:
# 分组迭代
df3 = pd.DataFrame({'X': ['A', 'B', 'A', 'B'], 'Y': [1, 4, 3, 2]})
g = df3.groupby('X')
g.groups
# for k, g in df3.groupby('X'):
#     print(k)
#     print(g)

{'A': Int64Index([0, 2], dtype='int64'),
 'B': Int64Index([1, 3], dtype='int64')}

### transform 操作

In [52]:
company=["A","B","C"]

data=pd.DataFrame({
    "company":[company[x] for x in np.random.randint(0,len(company),10)],
    "salary":np.random.randint(5,50,10),
    "age":np.random.randint(15,50,10)
}
)
data

,age,company,salary
0,33,C,45
1,19,A,40
2,43,C,20
3,26,B,39
4,24,A,8
5,19,C,18
6,22,B,22
7,26,A,23
8,21,B,12
9,26,A,27


In [53]:
group = data.groupby("company")  # <pandas.core.groupby.groupby.DataFrameGroupBy object at 0x10598fe90>

for key ,g in group:
    print('key: %s' % key)
    print(g)
    print('\n')

key: A
   age company  salary
1   19       A      40
4   24       A       8
7   26       A      23
9   26       A      27


key: B
   age company  salary
3   26       B      39
6   22       B      22
8   21       B      12


key: C
   age company  salary
0   33       C      45
2   43       C      20
5   19       C      18




In [54]:
# 新增一列记录各公司薪水均值
X = data.groupby('company')['salary'].mean().to_dict()
data['avg_salary'] = data['company'].map(X)
data.head()

,age,company,salary,avg_salary
0,33,C,45,27.666667
1,19,A,40,24.500000
2,43,C,20,27.666667
3,26,B,39,24.333333
4,24,A,8,24.500000


In [55]:
# 使用transform的形式
data['avg_salary'] = data.groupby('company')['salary'].transform('mean')
data.head()

,age,company,salary,avg_salary
0,33,C,45,27.666667
1,19,A,40,24.500000
2,43,C,20,27.666667
3,26,B,39,24.333333
4,24,A,8,24.500000


### apply
group之后apply，以分组后的子DataFrame作为参数传入指定函数的，基本操作单位是DataFrame, 返回值也是一个df

参考：https://juejin.im/post/5e1a80c3e51d45020078761f

In [56]:
def get_oldest(df):
    df.sort_values(by='age', ascending=True)
    return df.iloc[-1,:]

data.groupby('company').apply(get_oldest)

,age,company,salary,avg_salary
company,,,,
A,26,A,27,24.500000
B,21,B,12,24.333333
C,19,C,18,27.666667


### 获取分组里最大值所在的行方法

In [70]:
df = pd.DataFrame({'Sp':['a','b','c','d','e','f'], 'Mt':['s1', 's1', 's2','s2','s2','s3'], 'Value':[1,2,3,4,5,6], 'Count':[3,2,5,10,10,6]})
df

,Count,Mt,Sp,Value
0,3,s1,a,1
1,2,s1,b,2
2,5,s2,c,3
3,10,s2,d,4
4,10,s2,e,5
5,6,s3,f,6


In [71]:
# 先按Mt列进行分组，然后对分组之后的数据框使用idxmax函数取出Count最大值所在的列，再用iloc位置索引将行取出
df.iloc[df.groupby(['Mt']).apply(lambda x: x['Count'].idxmax())]

,Count,Mt,Sp,Value
0,3,s1,a,1
3,10,s2,d,4
5,6,s3,f,6


### 组内排序
- sort_values 方法
- groupby with sort_values

In [75]:
df = pd.DataFrame([['A',1],['A',3],['A',2],['B',5],['B',9]], columns = ['name','score'])
df

,name,score
0,A,1
1,A,3
2,A,2
3,B,5
4,B,9


In [77]:
df.sort_values(['name','score'], ascending = [True,False])

,name,score
1,A,3
2,A,2
0,A,1
4,B,9
3,B,5


In [82]:
df.groupby('name').apply(lambda d: d.sort_values('score', ascending=False)).reset_index(drop=True)

,name,score
0,A,3
1,A,2
2,A,1
3,B,9
4,B,5
